# Tutorial de uso para crear scripts y notebooks de python (VQE)

La idea de este tutorial es indicar el flujo de trabajo para poder usar las funcionalidad definidas. Todo lo que se muestra a continuacion puede ser usado para scripts o para trabajar en jupyters. En todo lo que sigue a continuacion, se trabajara con estructuras moleculares (pero la idea es la misma para otros modelos, solo hay que cambiar las clases).

Hecho por: Javier Norambuena Leiva

In [1]:
import quantumsim as qs
import numpy as np
import pennylane as qml

## Definir sistema

El primer paso es definir el sistema a trabajar, dependiendo del sistema con el que se quiere trabajar se necesitaran parametros distintos (mas información en la wiki del repositorio). Para este ejemplo, utilizaremos una estructura molecular con parametros basicos, existen mas parametros pero para simplicar usaremos estos.

In [ ]:
symbols = ["N", "C"]
coordinates = np.array([0.0, 0.0, 2.5, 0.0, 0.0, 0.0])
params = {
    'mapping': "jordan_wigner",
    'charge': 0, 
    'mult': 1,
    'basis': 'sto-3g',
    'method': 'dhf',
}

system = qs.vqe_molecular(symbols, coordinates, params)

La variable **system** almacena la clase del hamiltoniano donde se tiene toda la información del sistema y la funcion de coste.

## Definir ansatz

Al estar trabajando con una estructura molecular, los dos mejores ansatz son el UCCSD y el k-UpCCGSD. Como ambas ansatz son similares, sus parametros son los mismos, pero hay que tener en cuenta que esto no siempre ocurre.

Al iniciar esta clase, no existe un constructor base, se necesita llamar de forma individual cada metodo para asignar las variables de las clases.

In [ ]:
ansatz_params = {
    "repetitions": 1,
    "base": "default.qubit",
    "interface": "autograd",
    "electrons": 13,
}

UCCSD_ansatz = qs.uccds_ansatz()
UCCSD_ansatz.set_device( ansatz_params )
UCCSD_ansatz.set_node( ansatz_params )
UCCSD_ansatz.set_state( ansatz_params["electrons"] ) 

In [ ]:
ansatz_params = {
    "repetitions": 1,
    "base": "default.qubit",
    "interface": "autograd",
    "electrons": 13,
    }

kUp_ansatz = qs.upccgsd_ansatz()
kUp_ansatz.set_device( ansatz_params )
kUp_ansatz.set_node( ansatz_params )
kUp_ansatz.set_state( ansatz_params["electrons"] ) 

a,b = qml.kUpCCGSD.shape(k=ansatz_params["repetitions"], n_wires=system.qubits, delta_sz=0)

## Definir optimizador

El ultimo paso es construir la clase del optimizador, de momento, el unico que esta implementado es el de base en gradiente de pennylane. Lo unico que varia entre las clases definidas abajo es el numero de parametros, este numero de parametros esta definido en base al tipo de ansatz, por lo tanto, dependiendo del mismo se tendre un numero distinto. Fuera de este detalle, los parametros son los mismos.

In [ ]:
minimizate_params = {
    "maxiter": 100,
    "tol": 1e-6,
    "number":  len(UCCSD_ansatz.singles) + len(UCCSD_ansatz.doubles),
    "theta":["generic", 0.25]}

system.node = UCCSD_ansatz.node
optimizer = qs.gradiend_optimizer( minimizate_params )
energy1, optimum = optimizer.VQE( system.cost_function )

In [ ]:
minimizate_params = {
    "maxiter": 100,
    "tol": 1e-6,
    "number":  a*b,
    "theta":["generic", 0.25]}

system.node = kUp_ansatz.node
optimizer = qs.gradiend_optimizer( minimizate_params )
energy1, optimum = optimizer.VQE( system.cost_function )

Una vez construida la clase, lo siguiente es ejecutar uno de los metodos, definidos en la clase del optimizador, ingresando la funcion de coste (las cuales son definidas en cada clase del hamiltoniano).


Este es el esquema final de calculo para ejecutar el VQE, como se pudo ver, gran parte es solo construccion de las clases y el resto es ejecutado por detras dentro de las clases. 

Si esto quiere ser pasado a un script, solo hay que tomar el codigo de cada celda y dejarlo en un mismo archivo de python.

# Tutorial de uso para crear scripts y notebooks para ansatz adaptativo

El siguiente tutorial es para indicar como utilizar el adapt-VQE en el flujo de calculo del VQE.

Este proceso se tiene que dividir en dos pasos:

1. Construir el circuito minimo que funcionara como ansatz
2. Ejecutar el VQE


In [ ]:
import quantumsim as qs
import numpy as np
import pennylane as qml

## Construir el circuito minimo

In [ ]:
symbols = ["N", "C"]
coordinates = np.array([0.0, 0.0, 2.5, 0.0, 0.0, 0.0])
params = {
    'mapping': "jordan_wigner",
    'charge': 0, 
    'mult': 1,
    'basis': 'sto-3g',
    'method': 'dhf',
}

system = qs.adap_molecular(symbols, coordinates, params)

In [ ]:
ansatz_params = {
            "repetitions": 1,
            "base": "default.qubit",
            "interface": "autograd",
            "electrons": 13,
        }

system.set_device( ansatz_params )
system.set_state( ansatz_params["electrons"] )
system.set_node( ansatz_params )

In [ ]:
minimizate_params = {
    "maxiter": 20,
    "qubits": system.qubits,
    "electrons": 13,
    "theta":["generic", 0.3]}

optimizer = qs.adap_optimizer(minimizate_params)
minsingles, mindoubles = optimizer.MinimumCircuit(system.node)

Lo que obtenemos de este proceso es un conjunto de operadores de una y dos excitaciones los cuales deben ser usados para construir el ansatz final.

Para usar estos operadores, se tiene que usar el ansatz ''custom'', el cual permite definir una cantidad arbitraria de estos operadores para construir el ansatz.

## Ejecutar VQE con el circuito minimo

Ahora que tenemos el circuito de minimo tamañan, lo siguiente es usar esa descripcion pra construir un ansatz para calcular el estado de minima energia.

Usando la misma descipcion de symbols y coordinates, procedemos a construir el objeto de hamiltoniano para ser utilizado en el VQE.

In [ ]:
system = qs.vqe_molecular(symbols, coordinates, params)

Una de las cosas que varia del flujo del tutorial anterior es el tener que usar **obligatoriamente** la clase **custom_ansatz**, ya que esta es la unica que tiene metodos para cambiar las compuertas de exitaciones simples y dobles.

In [ ]:
ansatz_params = {
    "repetitions": 1,
    "base": "default.qubit",
    "interface": "autograd",
    "electrons": 13,
    "qubits": system.qubits,
}

ansazt = qs.custom_ansatz()
ansazt.set_device( ansatz_params )
ansazt.set_node( ansatz_params )
ansazt.set_state( ansatz_params["electrons"] )
ansazt.set_gates( singles=minsingles, doubles=mindoubles )

In [ ]:
minimizate_params = {
    "maxiter": 100,
    "tol": 1e-6,
    "number":  len(minsingles)+len(mindoubles),
    "theta":["generic", 0.25]}

system.node = ansazt.node
optimizer = qs.gradiend_optimizer(minimizate_params)
energy1, optimum = optimizer.VQE( system.cost_function )

Como se pudo ver, gran parte del flujo es exactamente el mismo que para el VQE, solo se agregan un par de cambios y se indica una clase especifica de ansatz para su correcto funcionamiento, el resto es exactamente lo mismo.

Este seria el flujo final para utilizar ansatz de tipo adaptativo, al igual que en el tutorial anterior, para llevarlo a un script, solo basta copias y pegar cada celda en el archivo de python seguiendo el orden expresado.

